# Final Project. Part 2. SQL

**Project background:**

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

**Tasks:**

- Find the number of books released after January 1, 2000.
- Find the number of user reviews and the average rating for each book.
- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
- Identify the author with the highest average book rating (look only at books with at least 50 ratings).
- Find the average number of text reviews among users who rated more than 50 books.

## Libraries, connection to data and functions

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Functions

In [3]:
# function for SQL queries
def sql(query):
    return pd.io.sql.read_sql(query, con = engine)

In [4]:
# function to download and check tables
def download (table):
    display(sql('SELECT * FROM ' + table + ' LIMIT 5')) # downloading 5 rows
    display(sql('SELECT COUNT(*) FROM ' + table)) # counting number of rows
   
    df=sql('SELECT * FROM ' + table) # checking for missing values and duplicates
    print('MISSING VALUES:') 
    print(df.isna().sum())
    print('')
    print('SHARE OF MISSING VALUES (%):')
    print(round(((df.isnull().sum()/len(df))*100),2))
    print('')
    print('DUPLICATES:')
    print(df.duplicated().sum())

## Studying the tables

### books

In [5]:
download ('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,count
0,1000


MISSING VALUES:
book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

SHARE OF MISSING VALUES (%):
book_id             0.0
author_id           0.0
title               0.0
num_pages           0.0
publication_date    0.0
publisher_id        0.0
dtype: float64

DUPLICATES:
0


### authors

In [6]:
download ('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,count
0,636


MISSING VALUES:
author_id    0
author       0
dtype: int64

SHARE OF MISSING VALUES (%):
author_id    0.0
author       0.0
dtype: float64

DUPLICATES:
0


### publishers

In [7]:
download ('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,count
0,340


MISSING VALUES:
publisher_id    0
publisher       0
dtype: int64

SHARE OF MISSING VALUES (%):
publisher_id    0.0
publisher       0.0
dtype: float64

DUPLICATES:
0


### ratings

In [8]:
download ('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,count
0,6456


MISSING VALUES:
rating_id    0
book_id      0
username     0
rating       0
dtype: int64

SHARE OF MISSING VALUES (%):
rating_id    0.0
book_id      0.0
username     0.0
rating       0.0
dtype: float64

DUPLICATES:
0


### reviews

In [9]:
download ('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,count
0,2793


MISSING VALUES:
review_id    0
book_id      0
username     0
text         0
dtype: int64

SHARE OF MISSING VALUES (%):
review_id    0.0
book_id      0.0
username     0.0
text         0.0
dtype: float64

DUPLICATES:
0


## Task 1: Find the number of books released after January 1, 2000.

In [10]:
sql(''' SELECT COUNT(publication_date)
             FROM books
             WHERE publication_date > '2000-01-01';
        ''')

,count
0,819


82% of books (819 out of 1000) released after 1 Jan 2000

## Task 2: Find the number of user reviews and the average rating for each book.

In [11]:
books = sql(''' SELECT b.book_id,
                    b.title,
                    rev.reviews_count,
                    AVG(rat.rating) AS rating_avg
             FROM books AS b
             JOIN ratings AS rat on b.book_id=rat.book_id
             JOIN (SELECT book_id,
                          COUNT(book_id) AS reviews_count
                   FROM reviews 
                   GROUP BY book_id) AS rev ON b.book_id=rev.book_id
             GROUP BY b.book_id, rev.reviews_count
             ORDER BY reviews_count  DESC             
             ''')
books.head(10)

,book_id,title,reviews_count,rating_avg
0,948,Twilight (Twilight #1),7,3.662500
1,207,Eat Pray Love,6,3.395833
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,497,Outlander (Outlander #1),6,4.125000
5,627,The Alchemist,6,3.789474
6,656,The Book Thief,6,4.264151
7,673,The Catcher in the Rye,6,3.825581
8,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
9,696,The Da Vinci Code (Robert Langdon #2),6,3.830508


In [12]:
books['reviews_count'].sum()

2793

In [13]:
books.describe()

,book_id,reviews_count,rating_avg
count,994.000000,994.000000,994.000000
mean,501.144869,2.809859,3.898699
std,288.845619,1.055873,0.562190
min,1.000000,1.000000,1.500000
25%,252.250000,2.000000,3.500000
50%,501.500000,3.000000,4.000000
75%,750.750000,3.000000,4.333333
max,1000.000000,7.000000,5.000000


- 994 out of 1000 books in the dataset have reviews and ratings
- number of reviews varies from 1 to 7 with mean of 2.8
- average rating varies from 1.5 to 5.0 with mean of 3.9

## Task 3: Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [14]:
publishers = sql (''' SELECT b1.publisher_id, 
                    b1.publisher,
                    b1.num_books
                    FROM (SELECT b.publisher_id,
                         p.publisher,
                         COUNT(b.publisher_id) as num_books
                         FROM books as b
                         JOIN publishers as p ON b.publisher_id=p.publisher_id
                         WHERE b.num_pages > 50
                         GROUP BY b.publisher_id, p.publisher
                         ORDER BY num_books DESC
                         LIMIT 1) AS b1
                   ''')
publishers

,publisher_id,publisher,num_books
0,212,Penguin Books,42


Publisher "Penguin Books" released greatest number of books - 42 (4% of total number of books in the dataset)

## Task 4: Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [15]:
sql(''' SELECT AVG(b1.rating_avg_book) AS rating_avg_auth, 
                    a.author
             FROM (SELECT b.book_id,
                          b.author_id,
                          AVG(rat.rating) AS rating_avg_book,
                          COUNT(rating)
                   FROM books AS b
                   JOIN ratings AS rat on b.book_id=rat.book_id
                   GROUP BY b.book_id
                   HAVING COUNT(rating) > 50) AS b1
             JOIN authors AS a ON b1.author_id=a.author_id
             GROUP BY b1.author_id, a.author
             ORDER BY rating_avg_auth DESC
             LIMIT 1            
      ''')

,rating_avg_auth,author
0,4.283844,J.K. Rowling/Mary GrandPré


The author with the highest average book rating (for books with at least 50 ratings) is J.K.Rowling/Mary GrandPré.

## Task 5: Find the average number of text reviews among users who rated more than 50 books.

In [16]:
sql(''' SELECT AVG(t1.num_reviews) 
              FROM (SELECT username,
                     COUNT(username) as num_reviews
                     FROM reviews as rev
                     WHERE rev.username IN (SELECT username                                                                    
                                           FROM ratings as rat
                                           GROUP BY username
                                           HAVING COUNT(username) > 50) 
                    GROUP BY username) as t1
     ''')

,avg
0,24.333333


The users who rated more than 50 books, have written 24 text reviews on average.

## Conclusion

Using SQL queries we found out the following:
- 82% of books (819 out of 1000) released after 1 Jan 2000
- 994 out of 1000 books in the dataset have reviews and ratings
    - number of reviews varies from 1 to 7 with mean of 2.8
    - average rating varies from 1.5 to 5.0 with mean of 3.9
- Publisher "Penguin Books" released greatest number of books - 42 (4% of total number of books in the dataset)
- The author with the highest average book rating (for books with at least 50 ratings) is J.K.Rowling/Mary GrandPré
- The users who rated more than 50 books, have written 24 text reviews on average

This data combined with other information can be used to develop new application for book readers.